##Requirements
Before running this notebook make sure you have the following:
- An Azure ML workspace
- A compute instance

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [49]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice
from azureml.data.dataset_type_definitions import PromoteHeadersBehavior
from azureml.widgets import RunDetails

import requests
import json
import pandas as pd
import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
#Retrieve current workspace
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Tuscan_Wine_Forecast'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')



Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F8L7P7LK9 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-162212
Azure region: southcentralus
Subscription id: d7f39349-a66b-446e-aba6-0053c2cf1c11
Resource group: aml-quickstarts-162212


## Compute Cluster
If compute cluster already exists, it will just identify it and end.


In [3]:
#Create Cluster if does not already exist

cluster_name = "myCluster"
try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Cluster already created")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_DS12_V2",min_nodes=1, max_nodes=6)
    cluster = ComputeTarget.create(ws,cluster_name, compute_config) #creates the actual cluster

cluster.wait_for_completion(show_output=True)  #Allows to continue on other threads while cluster is being created


#SOURCE / HELP: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute.amlcompute?view=azure-ml-py

InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset
A Dataset is created from the csv file in the folder "data".

In [45]:
# Create a dataset from delimited files with header

#Retrieves default Datastore and display info
datastore = ws.get_default_datastore() 
print(datastore)

#Google Drive path to dl csv
url_data = "https://docs.google.com/spreadsheets/d/1X9M3eNuBDv0ZKsOdidkdaBx9W1NubNDz3kxXuULsXmo/export?format=csv"

#Loads into Dataframe
df = pd.read_csv(url_data)

#Save into Datastore AND Register as dataset
dataset = Dataset.Tabular.register_pandas_dataframe(df,datastore,'Dataset_Wine',description="A Dataset composed of Tuscan wines with ratings, vintage, grapes and corresponding season avg weather")


#Display top 10 lines
dataset.take(10).to_pandas_dataframe()


#----Previous exploration to upload file from within Azure

#directory = os.getcwd()
#print(directory)
#datastore.upload(src_dir='./data',
#                  target_path='datasets/',
#                  overwrite=True)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-e7bb20a4-b569-4f6c-a2f5-5ce440eecdf6",
  "account_name": "mlstrg162212",
  "protocol": "https",
  "endpoint": "core.windows.net"
}
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162212/code


Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/e1625863-9fea-476a-91cc-9260248c0a79/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


,vintage,points,variety,winery,avg winter temp,avg spring temp,avg summer temp,avg fall temp,avg winter sun hour,avg spring hour,avg summer sun hour,avg fall sun hour,avg daily precip winter,avg daily precip spring,avg daily precip summer,avg daily precip fall
0,2010,85,Sangiovese,Casa Raia,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
1,2010,89,Sangiovese,Molinari Carlo,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
2,2010,92,Sangiovese,Castelli del Grevepesa,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
3,2010,93,Sangiovese,Le Gode,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
4,2010,97,Cabernet Franc,Le Macchiole,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
5,2010,90,Red Blend,Toscolo,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
6,2010,88,Sangiovese Grosso,Tenute Silvio Nardi,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
7,2010,88,Sangiovese,Mocali,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
8,2010,88,Chardonnay,Vigliano,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538
9,2010,93,Red Blend,Boscarelli,4.661111,13.25,25.271739,15.082418,6.451111,11.759783,14.097826,8.967033,2.06,1.720652,1.076087,1.461538


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [46]:
# TODO: Put your automl settings here
#automl_settings = {}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data= dataset,
    validation_size = 0.33,
    label_column_name='points',
    compute_target = cluster
    )

In [47]:
# TODO: Submit your experiment
runAutoML = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
Tuscan_Wine_Forecast,AutoML_feed62b9-2f1e-47db-856f-6ec1ad1c56f0,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [50]:
RunDetails(runAutoML).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [54]:
best_run, fitted_model = runAutoML.get_output()
print(best_run)



#print_model(fitted_model)

RunDetails(best_run).show()

Run(Experiment: Tuscan_Wine_Forecast,
Id: AutoML_feed62b9-2f1e-47db-856f-6ec1ad1c56f0_30,
Type: azureml.scriptrun,
Status: Completed)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

2021-10-27 18:22:13.443658: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-10-27 18:22:13.443706: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [55]:
#TODO: Save the best model

# Register run as Model

model_name = "AleaumeModelAutoML"
description = "Best AutoML Model"
model_path ="outputs/modelAutoML.pkl"

#model = best_run.register_model(model_name = model_name, description = description, model_path = model_path )

model = runAutoML.register_model(model_name = model_name, description = description)

#SOURCE / HELP: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python

#Unfortunately so far unsuccessful to register the model via SDK and get a default generated swagger.json uri. Also,  MSFT support recommends to do so via AML studio...

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [56]:
#To create a local deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=9064)

#Create Environment

env = Environment(name="AzureML-Tutorial")
myenv=env.clone("myenv")
myenv.python.conda_dependencies.add_pip_package("joblib==1.1.0")

my_inference_config = InferenceConfig(
    environment=env,
    source_directory="./",
    entry_script="./score_dummy.py"
    #entry_script="./score.py"
)

# Deploy the service locally

service = model.deploy(ws, "local-service", [model], my_inference_config, deployment_config)
service.reload()
print(service.get_logs())

print(service.scoring_uri)

service.wait_for_deployment(show_output=True)


ERROR:azureml._model_management._util:entry_script /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162212/code/./score_dummy.py doesn't exist. entry_script should be path relative to current working directory

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


WebserviceException: WebserviceException:
	Message: entry_script /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162212/code/./score_dummy.py doesn't exist. entry_script should be path relative to current working directory
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "entry_script /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook162212/code/./score_dummy.py doesn't exist. entry_script should be path relative to current working directory"
    }
}

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#Call model to test 

service.update(enable_app_insights=True)


uri = service.scoring_uri
requests.get("http://localhost:9064")
headers = {"Content-Type": "application/json"}
data = {
    "query": "This is a test query"
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

# Deleting the webservice


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
